In [67]:
from phasic_tonic.PhasicTonic import PhasicTonic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup    
from phasic_tonic.utils import get_sequences, preprocess

import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
OUTPUT_DIR1 = "/home/nero/phasic_tonic/data/analysis_output/whole_posttrial5/"
OUTPUT_DIR2 = "/home/nero/phasic_tonic/data/analysis_output/segmented_posttrial5/"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

per_trial_stats = []
per_epochs_stats = {}

def analysis(lfp, hypno, metadata):
    # Phasic tonic analysis
    g = PhasicTonic(fs=targetFs, thr_dur=900)
    g.fit(lfp, hypno)
    df = g.compute_stats()

    # Save the metadata and results
    for key in reversed(metadata):
        df.insert(0, column=key, value=metadata[key])
    
    per_trial_stats.append(df)

    # Save duration bouts
    for state, interval in [("phasic", g.phasic_interval), ("tonic", g.tonic_interval)]:
        for i, duration in enumerate(np.diff(interval, 1)):
            for condition in metadata.keys():
                per_epochs_stats[condition].append(metadata[condition])
            per_epochs_stats['state'].append(state)
            per_epochs_stats['epoch_id'].append(i)
            per_epochs_stats['duration'].append(duration.item())
        

In [68]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

In [73]:
per_trial_stats = []
per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'state' : [],
    'duration' : []
}

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, _ = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
        
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()

        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Preprocess
        lfpHPC_down = preprocess(lfpHPC, n_down, targetFs)
        analysis(lfpHPC_down, hypno, metadata)

df_epochs = pd.DataFrame(per_epochs_stats)
df_trial = pd.concat(per_trial_stats, axis=0)

  0%|          | 0/539 [00:00<?, ?it/s]

/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/phasic_tonic/src/phasic_tonic/PhasicTonic.py:93: UserWarning: Some epochs have no duration
  self.rem_interval = nap.IntervalSet(rem_start, rem_end)
/home/nero/phasic_tonic/src/phasic_tonic/PhasicTonic.py:232: RuntimeWarning: Mean of empty slice.
  stats["mean_duration"].append(durations.mean())
/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(r

In [74]:
df_trial

,rat_id,study_day,condition,treatment,trial_num,rem_start,rem_end,state,num_bouts,mean_duration,total_duration,percent_of_rem
0,5,8,HC,0,3,1331,1485,phasic,3,1.958000,5.874,0.038143
1,5,8,HC,0,3,1331,1485,tonic,4,37.031500,148.126,0.961857
2,5,8,HC,0,3,2428,2475,phasic,1,3.154000,3.154,0.067106
3,5,8,HC,0,3,2428,2475,tonic,2,21.923000,43.846,0.932894
0,5,8,HC,0,2,411,496,phasic,4,1.664000,6.656,0.078306
...,...,...,...,...,...,...,...,...,...,...,...,...
5,11,4,OR,4,3,928,1012,tonic,2,41.295000,82.590,0.983214
6,11,4,OR,4,3,1791,1928,phasic,5,1.700800,8.504,0.062073
7,11,4,OR,4,3,1791,1928,tonic,6,21.416000,128.496,0.937927
8,11,4,OR,4,3,1961,2026,phasic,2,2.463000,4.926,0.075785


In [75]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,state,duration
0,5,8,HC,0,3,0,phasic,1.786
1,5,8,HC,0,3,1,phasic,1.460
2,5,8,HC,0,3,2,phasic,2.628
3,5,8,HC,0,3,3,phasic,3.154
4,5,8,HC,0,3,0,tonic,48.984
...,...,...,...,...,...,...,...,...
8306,11,4,OR,4,3,11,tonic,24.730
8307,11,4,OR,4,3,12,tonic,3.412
8308,11,4,OR,4,3,13,tonic,45.264
8309,11,4,OR,4,3,14,tonic,5.040


In [78]:
df_trial.to_csv(OUTPUT_DIR1+"phasic_tonic_per_rem_epochs.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR1+"phasic_tonic_per_durations.csv", index=False)